In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
raw_data = []
# Load dữ liệu từ file txt
try:
    with open("../data/raw/INTL.txt", "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                try:
                    json_obj = json.loads(line)
                    raw_data.append(json_obj)
                except json.JSONDecodeError as e:
                    print(e)
except Exception as e:
    print(e)


In [3]:
df = pd.DataFrame(raw_data)

In [4]:
# Lấy các hàng dữ liệu về tổng năng lượng tiêu thụ của các nuớc
data = df[
    df["name"].str.contains("Total energy consumption", regex=False)
    & (df["units"] == "terajoules")
].reset_index(drop=True)


In [5]:
# Lấy các cột cần thiết
data = data.loc[:, ["name", "geography", "data"]]

In [6]:
# Loại bỏ các hàng không có data
data = data.dropna(subset=["data"])

In [7]:
# Extract tên country từ cột name
data["name"] = data["name"].str.extract(r"Total energy consumption, (.+?), Annual")
data = data.rename(columns={"name": "country"})

In [8]:
# Loại bỏ dữ liệu trùng lặp theo country
data = data.drop_duplicates(subset="country", keep="last")

In [9]:
# tách mảng 2 chiều trong cột data thành nhiều hàng
data = data.explode("data").reset_index(drop=True)

In [10]:
data["year"] = data["data"].str[0]
data["data"] = data["data"].str[1]
data = data.rename(columns={"data": "value"})

In [11]:
data["value"] = data["value"].replace("--", np.nan).astype(float)

/tmp/ipykernel_22559/4094537107.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["value"] = data["value"].replace("--", np.nan).astype(float)
